In [ ]:
import os
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from yt_dlp import YoutubeDL
import whisper
import threading  # For running transcription in the background

In [ ]:
def download_video():
    url = url_entry.get()
    resolution = resolution_var.get()
    audio_only = audio_var.get()
    output_folder = folder_path.get()

    if not url or not output_folder:
        messagebox.showerror("Error", "Please provide a URL and output folder.")
        return

    ydl_opts = {
        'format': f'bestaudio/best' if audio_only else f'bestvideo[height<={resolution}]+bestaudio/best',
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),
        'noplaylist': True
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        messagebox.showinfo("Success", "Download completed!")
    except Exception as e:
        messagebox.showerror("Error", f"Failed to download video: {e}")

    # Close the GUI after download
    app.destroy()

In [ ]:
def select_folder():
    folder_selected = filedialog.askdirectory()
    folder_path.set(folder_selected)

In [ ]:
def transcribe_callback():
    # Resolve the actual string value of the file path
    video_file_path = file_path_var.get()
    if not video_file_path or not os.path.exists(video_file_path):
        messagebox.showerror("Error", "Please provide a valid file path.")
        return

    # Call the transcription function with the correct file path
    try:
        transcription_file = transcribe_audio(video_file_path)
        messagebox.showinfo("Success", f"Transcription saved to: {transcription_file}")
    except Exception as e:
        messagebox.showerror("Error", str(e))
        
    # Close the GUI after download
    app.destroy()

In [ ]:
# GUI setup
app = tk.Tk()
app.title("Video Downloader")
app.geometry("500x400")

# Video Downloader Section
tk.Label(app, text="Video Downloader", font=("Arial", 14)).pack(pady=10)

tk.Label(app, text="Video URL:").pack(pady=5)
url_entry = tk.Entry(app, width=50)
url_entry.pack(pady=5)

tk.Label(app, text="Resolution (max):").pack(pady=5)
resolution_var = tk.StringVar(value="1080")
ttk.Combobox(app, textvariable=resolution_var, values=["144", "360", "480", "720", "1080"]).pack(pady=5)

audio_var = tk.BooleanVar()
tk.Checkbutton(app, text="Audio Only", variable=audio_var).pack(pady=5)

tk.Label(app, text="Output Folder:").pack(pady=5)
folder_path = tk.StringVar()
tk.Entry(app, textvariable=folder_path, width=40).pack(pady=5)
tk.Button(app, text="Browse", command=select_folder).pack(pady=5)

tk.Button(app, text="Download", command=download_video).pack(pady=20)

# Transcription Section
tk.Label(app, text="Audio Transcription", font=("Arial", 14)).pack(pady=20)

tk.Label(app, text="Select Video File:").pack(pady=5)
file_path_var = tk.StringVar()
tk.Entry(app, textvariable=file_path_var, width=50).pack(pady=5)
tk.Button(app, text="Browse", command=select_file).pack(pady=5)

tk.Button(app, text="Transcribe", command=transcribe_callback).pack(pady=20)

# Start the GUI main loop
app.mainloop()

In [ ]:
# Function to transcribe audio
def transcribe_audio(video_path):
    """Transcribe audio from a video file using Whisper."""
    if not os.path.exists(video_path):
        messagebox.showerror("Error", f"File not found: {video_path}")
        progress_label.config(text="Error: File not found.")
        return
    
    try:
        # Update progress
        progress_label.config(text="Loading Whisper model...")
        progress_bar["value"] = 10
        root.update()

        # Load Whisper model
        model = whisper.load_model("base")
        progress_label.config(text="Whisper model loaded successfully.")
        progress_bar["value"] = 30
        root.update()

        # Start transcription
        progress_label.config(text="Transcribing audio...")
        progress_bar["value"] = 60
        root.update()
        result = model.transcribe(video_path)
        progress_label.config(text="Transcription completed.")
        progress_bar["value"] = 90
        root.update()

        # Save transcription
        transcription_path = video_path.rsplit('.', 1)[0] + ".txt"
        with open(transcription_path, "w", encoding="utf-8") as f:
            f.write(result["text"])
        progress_label.config(text=f"Transcription saved to: {transcription_path}")
        progress_bar["value"] = 100
        root.update()

        messagebox.showinfo("Success", f"Transcription saved to:\n{transcription_path}")
    except Exception as e:
        progress_label.config(text="Error during transcription.")
        messagebox.showerror("Error", f"An error occurred: {e}")
        return
    
    # Close the GUI after transcription
    root.destroy()
    

In [ ]:
# Function to start transcription in a separate thread
def start_transcription():
    """Run transcription in a separate thread to keep the GUI responsive."""
    video_path = file_path_var.get()
    if not video_path:
        messagebox.showerror("Error", "Please select a video file first.")
        return

    # Reset progress bar and label
    progress_bar["value"] = 0
    progress_label.config(text="Starting transcription...")

    # Run transcription in a separate thread
    threading.Thread(target=transcribe_audio, args=(video_path,), daemon=True).start()

In [ ]:
# File selection function
def select_file():
    """Open a file dialog to select a video file for transcription."""
    file_path = filedialog.askopenfilename(
        title="Select Video File",
        filetypes=[("Video Files", "*.mp4 *.mkv *.avi")]
    )
    if file_path:
        file_path_var.set(file_path)  # Update the Tkinter StringVar with the selected file path
        progress_label.config(text="File selected. Ready to transcribe.")

In [ ]:
# GUI setup
root = tk.Tk()
root.title("Whisper Transcriber with Progress Bar")
root.geometry("500x300")

file_path_var = tk.StringVar()  # Variable to store the selected file path

# File selection
tk.Label(root, text="Select Video File:").pack(pady=5)
tk.Entry(root, textvariable=file_path_var, width=50, state="readonly").pack(pady=5)
tk.Button(root, text="Browse", command=select_file).pack(pady=5)

# Progress bar and label
progress_bar = ttk.Progressbar(root, orient="horizontal", length=400, mode="determinate")
progress_bar.pack(pady=20)
progress_label = tk.Label(root, text="No task running.")
progress_label.pack(pady=5)

# Transcribe button
tk.Button(root, text="Transcribe", command=start_transcription).pack(pady=20)

root.mainloop()